In [1]:
import yaml
import logging


with open("docker-compose.yml", "r") as file:
    yml = yaml.safe_load(file)

class DBCred():
    def __init__(self, service) -> None:
        try:
            db =  yml["services"][service]
            env = db["environment"]

            if service == "pg_db":
                self.db_name = env["POSTGRES_DB"]
                self.db_prefix = "POSTGRES"
            if service == "mysql_db":
                self.db_name = env["MYSQL_DATABASE"]
                self.db_prefix = "MYSQL"

            self.dsn =  f"""
                host={"localhost"}
                dbname={self.db_name}
                user={env[f"{self.db_prefix}_USER"]}
                password={env[f"{self.db_prefix}_PASSWORD"]}
                port={yml["services"][service]["ports"][0].split(":")[0]}
            """
        except Exception as e:
            logging.error(f"Invalid docker-compose.yml: {e}")

In [11]:

source_db_cred = DBCred("pg_db")
destiny_db_cred = DBCred("mysql_db")

In [9]:
import psycopg2

In [12]:
class Database():
    def __init__(self, db_cred):
        self.name = db_cred.db_name
        self.credentials_dsn = db_cred.dsn
        self.conn = self.connect()
        self.cur = self.conn.cursor()
    
    def connect(self):
        try:
            return psycopg2.connect(self.credentials_dsn)
        except Exception as error:
            print("erro")

    def get_table_names(self, cursor, sql):
        """
            Retrieve name of tables from source DB
            return 'list' : 'table_names'
        """
        try:
            cursor.execute(sql)
            results = cursor.fetchall()
            table_names = [table_name[0] for table_name in results]
            print(f"Got {len(table_names)} tables: {table_names} \n")
            return table_names
        except Exception as e:
            print(f"Error retrieving table names, check database status or SQL syntax: \n{e}")

    def close_conn(self):
        self.cur.close()
        self.conn.close()



In [13]:
source_db = Database(db_cred=source_db_cred)

In [15]:
source_db.get_table_names()

TypeError: Database.get_table_names() missing 2 required positional arguments: 'cursor' and 'sql'

In [10]:
conn = psycopg2.connect(source_db_cred)
cursor = conn.cursor()

TypeError: Expected bytes or unicode string, got DBCred instead